# Random polymer
This example shows you how to run a simple polymer simulation in hoomd-blue.
http://glotzerlab.engin.umich.edu/hoomd-blue/

Here is a script that generates a system of bead-spring polymers that self-assemble into a hex phase when run for a few million time steps. The polymers are A6B7A6 block copolymers in an implicit solvent. The script also shows a few examples of how writing python code in the script can be handy: here the concentration phi_P is a parameter and math operations are performed to calculate the length of the box.

For more information on the model in this script, see "Micellar crystals in solution from molecular dynamics simulations" J. Chem. Phys. 128, 184906 (2008); DOI:10.1063/1.2913522 http://link.aip.org/link/?JCPSA6/128/184906/1

Any of the polymer systems in the paper could be easily run just by changing a few parameters in this script. from: https://lost-contact.mit.edu/afs//umich.edu/user/j/o/joaander/Public/hoomd-web/doc/page_example_scripts.html

## Initialize
First import hoomd and associated libraries and then initialize. If no mode is given then the fastest processor cpu or gpu will be selected automatically.

In [1]:
#import hoomd
import hoomd.md
from hoomd import *
from hoomd.deprecated import *

#context.initialize();
#context.initialize("--mode=cpu");
context.initialize("--mode=gpu");

HOOMD-blue v2.2.2-29-g1e86eec CUDA (8.0) DOUBLE HPMC_MIXED MPI SSE SSE2 
Compiled: 01/29/18
Copyright 2009-2017 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
notice(2): This system is not compute exclusive, using local rank to select GPUs
notice(2): Unable to identify node local rank information
notice(2): Using global rank to select GPUs
HOOMD-blue is running on the following GPU(s):
 [0]  GeForce GTX 1060 6GB  10 SM_6.1 @ 1.84 GHz, 6075 MiB DRAM, DIS


## Import other libraries and define parameters

In [2]:
import math
# parameters
phi_P = 0.25
n_poly = 1

## Define the polymer

In [3]:
polymer1 = dict(bond_len=1.2, type=['A']*300 + ['B']*400 + ['A']*300,bond="linear", count=n_poly)
# perform some simple math to find the length of the box
N = len(polymer1['type']) * polymer1['count']

# generate the polymer system
init.create_random_polymers(box=data.boxdim(volume=10*math.pi * N / (6.0 * phi_P)), polymers=[polymer1],separation=dict(A=0.35, B=0.35),seed=12)

notice(2): Group "all" created containing 1000 particles


## Setup the bonds and force fields

In [4]:
# force field setup
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=330.0, r0=0.84)
nl = hoomd.md.nlist.cell();
lj = hoomd.md.pair.lj(r_cut=3.0,nlist=nl)
#nl = hoomd.md.nlist.tree();
#lj = hoomd.md.pair.lj(r_cut=3.0,nlist=nl)
lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0, alpha=0.0,r_cut=2**(1.0/6.0))
lj.pair_coeff.set('A', 'B', epsilon=1.0, sigma=1.0, alpha=0.0,r_cut=2**(1.0/6.0))
lj.pair_coeff.set('B', 'B', epsilon=1.0, sigma=1.0, alpha=1.0,r_cut=3.0)
lj.set_params(mode='shift')

## Integrate the simulation

In [ ]:
# integrate NVT for a bunch of time steps
all = group.all()
hoomd.md.integrate.mode_standard(dt=0.005)
hoomd.md.integrate.nvt(group=all, kT=1.2, tau=0.5)

#save a gsd file at t=0
hoomd.dump.gsd(filename="gsd/random_polymer2.gsd", overwrite=True, period=None, group=group.all(), time_step=0)
# setup the IMD server
hoomd.analyze.imd(port=54321, period=100)
# run a very long time so the simulation can be watched in VMD
run(1e6)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 998
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
notice(2): analyze.imd: listening on port 54321
Time 00:00:10 | Step 82218 / 1000000 | TPS 8221.74 | ETA 00:01:51
Time 00:00:20 | Step 159066 / 1000000 | TPS 7684.79 | ETA 00:01:49
Time 00:00:30 | Step 227723 / 1000000 | TPS 6865.69 | ETA 00:01:52
notice(2): analyze.imd: accepted connection
Time 00:00:40 | Step 293625 / 1000000 | TPS 6590.13 | ETA 00:01:47
Time 00:00:50 | Step 373495 / 1000000 | TPS 7986.95 | ETA 00:01:18
Time 00:01:00 | Step 454496 / 1000000 | TPS 8100.01 | ETA 00:01:07
Time 00:01:10 | Step 534291 / 1000000 | TPS 7979.47 | ETA 00:00:58
Time 00:01:20 | Step 614150 / 1000000 | TPS 7985.89 | ETA 00:00:48
Time 00:01:30 | Step 701723 / 1000000 | TPS 8757.26 | ET

The results of this example may be visualized with VMD which can be installed from here: http://www.ks.uiuc.edu/Research/vmd/

While running must execute the following command in a terminal:

In [ ]:
#vmd -e imd.vmd
!cat imd.vmd

![](snapshots/copolymer.png)